# Carga Datasets 

En este Jupyter notebook insertamos el dataset original y el transformado en las tablas que creamos en el Jupyter notebook tablas_datasets

#### Cambiamos el directorio raíz del proyecto para facilitar la carga de los datasets y las importaciones

In [1]:
import os

notebook_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
os.chdir(root_dir)

print("Directorio de trabajo actual después del cambio:", os.getcwd())

Directorio de trabajo actual después del cambio: c:\Users\mirlo\proyecto1


#### Importamos las Librerias y la conexion

In [12]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from Database.conexion_BD import create_connection

connection = create_connection()

Conexión exitosa a la base de datos PostgreSQL.


#### Crear el engine de SQLAlchemy usando la conexión existente

In [13]:
postgresql_engine = create_engine('postgresql+psycopg2://', creator=lambda: connection)

#### Cargamos el Dataset Original 

In [14]:
ruta_dataset_original = 'csv/train.csv'
train = pd.read_csv(ruta_dataset_original, delimiter=';', on_bad_lines='skip')

#### Remplazamos las comas por puntos para que los datos con notacion cientifica no interrumpan este proceso, y ajustamos los nombres de las columnas para que coincidan con los nombres de la tabla en PostgreSQL 

In [15]:
if train is not None:
    for col in train.select_dtypes(include=['object']).columns:
        train[col] = train[col].str.replace(',', '.', regex=False)

    train.columns = [
        'month', 'day_of_week', 'dep_del15', 'dep_time_blk', 'distance_group', 'segment_number',
        'concurrent_flights', 'number_of_seats', 'carrier_name', 'airport_flights_month',
        'airline_flights_month', 'airline_airport_flights_month', 'avg_monthly_pass_airport',
        'avg_monthly_pass_airline', 'flt_attendants_per_pass', 'ground_serv_per_pass', 'plane_age',
        'departing_airport', 'latitude', 'longitude', 'previous_airport', 'prcp', 'snow', 'snwd',
        'tmax', 'awnd', 'carrier_historical', 'dep_airport_hist', 'day_historical', 'dep_block_hist'
    ]
else:
    print("No se pudo leer el archivo CSV, no se insertarán datos.")

#### Lo insertamos en la tabla (Devido a la cantidad de datos es normal que se demore un poco al ejecutar el codigo)

In [16]:
if train is not None and not train.empty:
    try:
        train.to_sql('flights', con=postgresql_engine, if_exists='append', index=False)
        print("Datos insertados en la base de datos exitosamente.")
    except Exception as e:
        print(f"Error al insertar datos: {e}")
else:
    print("El DataFrame está vacío, no se insertarán datos.")

Datos insertados en la base de datos exitosamente.
